In [ ]:
import os
import requests
from bs4 import BeautifulSoup
from typing import List
from dotenv import load_dotenv
from openai import OpenAI
import google.generativeai
import anthropic
import gradio as gr

load_dotenv()

openai = OpenAI()
claude = anthropic.Client()
google.generativeai.configure()

system_message = "You are a helpful assistant"

def message_gpt(prompt):
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt}
    ]

    completion = openai.chat.completions.create(
        model='gpt-4o-mini',
        messages=messages
    )

    return completion.choices[0].message.content


message_gpt("What is today's date?")

In [ ]:
def shout(text):
    return text.upper()


shout("hello")

In [ ]:
view = gr.Interface(fn=shout, inputs="textbox", outputs='textbox', allow_flagging='never')

view.launch(share=True)

In [ ]:
view = gr.Interface(
    fn=shout,
    inputs = [gr.Textbox(label="Your message:", lines=6)],
    outputs = [gr.Textbox(label="Response:", lines=8)],
    allow_flagging="never"
)

view.launch()

In [ ]:
view = gr.Interface(fn=shout, 
                    inputs=[gr.Textbox(label="Your message:", lines=6)],
                    outputs=[gr.Textbox(label="Response:", lines=8)],
                    allow_flagging="never")
view.launch()

In [ ]:
system_message = "You are a helpful assistant that responds in markdown"

view = gr.Interface(
    fn=message_gpt,
    inputs=[gr.Textbox(label="Your message:")],
    outputs=[gr.Markdown(label="Response:")],
    allow_flagging="never"
)
view.launch()

In [ ]:
def stream_gpt(prompt):
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt}
    ]

    stream = openai.chat.completions.create(
        model='gpt-4o-mini',
        messages=messages,
        stream=True
    )

    result = ""
    for chunk in stream:
        result += chunk.choices[0].delta.content or ""
        yield result




view = gr.Interface(
    fn=stream_gpt,
    inputs=[gr.Textbox(label="Your message:")],
    outputs=[gr.Markdown(label="Response:")],
    allow_flagging="never"
)

view.launch()

In [ ]:
def stream_claude(prompt):
    result = claude.messages.stream(
        model="claude-3-haiku-20240307",
        max_tokens=1000,
        temperature=0.7,
        system=system_message,
        messages=[
            {"role": "user", "content": prompt},
        ],
    )
    response = ""
    with result as stream:
        for text in stream.text_stream:
            response += text or ""
            yield response




def stream_model(prompt, model):
    if model == "GPT":
        result = stream_gpt(prompt)
    elif model == "Claude":
        result = stream_claude(prompt)
    else:
        raise ValueError("Unsupported model")
    for chunk in result:
        yield chunk

In [ ]:
view = gr.Interface(
    fn=stream_model,
    inputs=[gr.Textbox(label="Your message:"), gr.Dropdown(label="Model", choices=["GPT", "Claude"])],
    outputs=[gr.Markdown(label="Response:")],
    allow_flagging="never"
)
view.launch()

In [ ]:
class Website:
    url: str
    title: str
    text: str

    def __init__(self, url):
        self.url = url
        response = requests.get(url)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", 'style', 'img', 'input']):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"
    

system_prompt = "You are an assistant that analyzes the contents of a company website landing page \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown."


def stream_brochure(company_name, url, model, tone):
    prompt = f"Please generate a company brochure for {company_name}. Here is their landing page:\n"
    prompt += Website(url).get_contents()
    if model == "GPT":
        result = stream_gpt(prompt)
    elif model == "Claude":
        result = stream_claude(prompt)
    else:
        raise ValueError("Unsupported model")
    
    for chunk in result:
        yield chunk

    

In [ ]:
view = gr.Interface(
    fn=stream_brochure,
    inputs=[
        gr.Textbox(label="Company Name:"),
        gr.Textbox(label="Landing Page URL:"),
        gr.Dropdown(label="Select Model", choices=["GPT", "Claude"])],
        outputs=[gr.Markdown(label="Brochure:")],
        allow_flagging="never"
)

view.launch()

In [ ]:
system_prompt_template = "You are an assistant that analyzes the contents of a company website landing page \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown and in a {} tone."


def message_gpt(prompt, system_prompt):
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": prompt}
    ]

    completion = openai.chat.completions.create(
        model='gpt-4o-mini',
        messages=messages
    )

    return completion.choices[0].message.content

def stream_claude(prompt, system_prompt):
    result = claude.messages.stream(
        model="claude-3-haiku-20240307",
        max_tokens=1000,
        temperature=0.7,
        system=system_prompt,
        messages=[
            {"role": "user", "content": prompt},
        ],
    )
    response = ""
    with result as stream:
        for text in stream.text_stream:
            response += text or ""
            yield response




def stream_model(prompt, model, tone):
    system_prompt = system_prompt_template.format(tone)

    if model == "GPT":
        result = stream_gpt(prompt, system_prompt)
    elif model == "Claude":
        result = stream_claude(prompt, system_prompt)
    else:
        raise ValueError("Unsupported model")

    for chunk in result:
        yield chunk


def stream_brochure(company_name, url, model, tone):
    prompt = f"Please generate a company brochure for {company_name}. Answer in a VERY {tone} tone. Here is their landing page:\n"
    print(prompt)
    prompt += Website(url).get_contents()
    if model == "GPT":
        result = stream_gpt(prompt)
    elif model == "Claude":
        result = stream_claude(prompt)
    else:
        raise ValueError("Unsupported model")
    
    for chunk in result:
        yield chunk


view = gr.Interface(
    fn=stream_brochure,
    inputs=[
        gr.Textbox(label="Company Name:"),
        gr.Textbox(label="Landing Page URL:"),
        gr.Dropdown(label="Select Model", choices=["GPT", "Claude"]),
        gr.Dropdown(label="Tone", choices=["Jovial", "Sad", "Rude"])
        ],
        outputs=[gr.Markdown(label="Brochure:")],
        allow_flagging="never"
)

view.launch()